Find Neighbors
Need to a road check. So its only traversable via roads

In [22]:
import pandas as pd
import numpy as np
import math

In [27]:
csv_file_path = "scats-10-2006.csv"

df_scats = pd.read_csv(csv_file_path)

df_scats_selected = df_scats[['SCATS Number', 'NB_LATITUDE', 'NB_LONGITUDE']]

unique_scats = df_scats_selected['SCATS Number'].unique()

In [30]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

all_data = []

for scats_n in unique_scats:
    row = df_scats_selected[df_scats_selected['SCATS Number'] == scats_n].iloc[0]
    scats_lat, scats_lon = row['NB_LATITUDE'], row['NB_LONGITUDE']
    
    min_distance = float('inf')
    closest_scats_n = None
    
    for other_scats_n in unique_scats:
        if scats_n != other_scats_n:
            other_row = df_scats_selected[df_scats_selected['SCATS Number'] == other_scats_n].iloc[0]
            other_lat, other_lon = other_row['NB_LATITUDE'], other_row['NB_LONGITUDE']
            
            distance = haversine(scats_lat, scats_lon, other_lat, other_lon)
            
            if distance < min_distance:
                min_distance = distance
                closest_scats_n = other_scats_n
    
    all_data.append([scats_n, scats_lat, scats_lon, closest_scats_n, min_distance, 60, 30])

processed = pd.DataFrame(all_data, columns=['SCATS Number', 'Latitude', 'Longitude', 'Closest SCATS', 'Distance (km)', 'Speed Limit (km/h)', 'Delay (seconds)'])

processed.to_csv('connections.csv', index=False)

processed.head()

,SCATS Number,Latitude,Longitude,Closest SCATS,Distance (km),Speed Limit (km/h),Delay (seconds)
0,970,-37.867030,145.091590,3685,1.388491,60,30
1,2000,-37.851683,145.094346,3685,0.335127,60,30
2,2200,-37.816310,145.098120,3126,1.277017,60,30
3,2820,-37.794770,145.030770,4335,1.332963,60,30
4,2825,-37.786610,145.062020,4030,1.001680,60,30
